In [1]:
import matplotlib.pyplot as plt
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from datetime import date, datetime
from multiprocessing.spawn import import_main_path
from keras.preprocessing.sequence import TimeseriesGenerator
from pandas_datareader import data
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import numpy as np

In [2]:
today = date.today()
# print(today)
start_date = '2010-01-01'
end_date = '2022-06-27'

panel_data_antm = data.DataReader('ANTM.JK','yahoo',start_date, end_date)
panel_data_asii = data.DataReader('ASII.JK','yahoo',start_date, end_date)
panel_data_icbp = data.DataReader('ICBP.JK','yahoo',start_date, end_date)
panel_data_jsmr = data.DataReader('JSMR.JK','yahoo',start_date, end_date)


In [3]:
#antm
panel_data_antm['average'] = panel_data_antm[['Open','Close','High','Low']].mean(axis=1).round(2)
panel_data_antm['daily_return'] = panel_data_antm['Open'] - panel_data_antm['Close']
#asii
panel_data_asii['average'] = panel_data_asii[['Open','Close','High','Low']].mean(axis=1).round(2)
panel_data_asii['daily_return'] = panel_data_asii['Open'] - panel_data_asii['Close']
#icbp
panel_data_icbp['average'] = panel_data_icbp[['Open','Close','High','Low']].mean(axis=1).round(2)
panel_data_icbp['daily_return'] = panel_data_icbp['Open'] - panel_data_icbp['Close']
#jsmr
panel_data_jsmr['average'] = panel_data_jsmr[['Open','Close','High','Low']].mean(axis=1).round(2)
panel_data_jsmr['daily_return'] = panel_data_jsmr['Open'] - panel_data_jsmr['Close']

In [4]:
df_antm = pd.DataFrame(panel_data_antm)
df_asii = pd.DataFrame(panel_data_asii)
df_icbp = pd.DataFrame(panel_data_icbp)
df_jsmr = pd.DataFrame(panel_data_jsmr)

In [5]:
# apply a min max scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_antm = pd.DataFrame(scaler.fit_transform(panel_data_antm),columns=['high','low','open','close','volume','adj_close','average','daily_return'])
df_asii = pd.DataFrame(scaler.fit_transform(panel_data_asii),columns=['high','low','open','close','volume','adj_close','average','daily_return'])
df_icbp = pd.DataFrame(scaler.fit_transform(panel_data_icbp),columns=['high','low','open','close','volume','adj_close','average','daily_return'])
df_jsmr = pd.DataFrame(scaler.fit_transform(panel_data_jsmr),columns=['high','low','open','close','volume','adj_close','average','daily_return'])


In [6]:
df_antm.drop(columns=['adj_close','volume'], inplace=True)
df_asii.drop(columns=['adj_close','volume'], inplace=True)
df_icbp.drop(columns=['adj_close','volume'], inplace=True)
df_jsmr.drop(columns=['adj_close','volume'], inplace=True)

In [7]:
# convert to windowed data sets
ylist_antm = panel_data_antm['Adj Close']
ylist_asii = panel_data_asii['Adj Close']
ylist_icbp = panel_data_icbp['Adj Close']
ylist_jsmr = panel_data_jsmr['Adj Close']


In [8]:
#LAGS and PERIOD
n_future = 20
n_past = 3*20
total_period = 4*20

# FUNGSI AKTIVASI

# ANTM

In [9]:
idx_end_antm = len(ylist_antm)
idx_start_antm = idx_end_antm - total_period

X_new_antm = []
y_new_antm = []

while idx_start_antm > 0:
  x_line_antm = ylist_antm[idx_start_antm:idx_start_antm+n_past]
  y_line_antm = ylist_antm[idx_start_antm+n_past:idx_start_antm+total_period]

  X_new_antm.append(x_line_antm)
  y_new_antm.append(y_line_antm)

  idx_start_antm = idx_start_antm - 1

X_new_antm = np.array(X_new_antm)
y_new_antm = np.array(y_new_antm)

# train test split
from sklearn.model_selection import train_test_split
X_train_antm, X_test_antm, y_train_antm, y_test_antm = train_test_split(X_new_antm, y_new_antm, test_size=0.33, random_state=42)

# reshape data into the right format for RNNs
n_samples = X_train_antm.shape[0]
n_timesteps = X_train_antm.shape[1]
n_steps = y_train_antm.shape[1]
n_features = 1

X_train_rs_antm = X_train_antm.reshape(n_samples, n_timesteps, n_features )
X_test_rs_antm = X_test_antm.reshape(X_test_antm.shape[0], n_timesteps, n_features )


In [28]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_antm_unit64_lr1_sigmoid = Sequential([
  LSTM(64, activation='sigmoid',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(64, activation='sigmoid',return_sequences=True),
  LSTM(64, activation='sigmoid'),
  Dense(y_train_antm.shape[1]),
])

simple_model_three_antm_unit64_lr1_sigmoid.summary()

simple_model_three_antm_unit64_lr1_sigmoid.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_antm_unit64_lr_sigmoid = simple_model_three_antm_unit64_lr1_sigmoid.fit(X_train_rs_antm, y_train_antm,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_three_antm_unit64_lr1_sigmoid = simple_model_three_antm_unit64_lr1_sigmoid.predict(X_test_rs_antm)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 60, 64)            16896     
                                                                 
 lstm_4 (LSTM)               (None, 60, 64)            33024     
                                                                 
 lstm_5 (LSTM)               (None, 64)                33024     
                                                                 
 dense_1 (Dense)             (None, 20)                1300      
                                                                 
Total params: 84,244
Trainable params: 84,244
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 10s 99ms/step - loss: 993.9017 - mae: 993.9017 - val_loss: 883.0706 - val_mae: 883.0706
Epoch 2/20
51/51 [========================

In [29]:
from sklearn.metrics import mean_absolute_percentage_error, r2_score

print("mae score antm: "+str(mean_absolute_error(preds_three_antm_unit64_lr1_sigmoid, y_test_antm).round(2)))
print("mape score antm: "+str(mean_absolute_percentage_error(preds_three_antm_unit64_lr1_sigmoid, y_test_antm).round(2)))
print("r2 score antm: "+str(r2_score(preds_three_antm_unit64_lr1_sigmoid, y_test_antm).round(2)))


mae score antm: 405.62
mape score antm: 0.45
r2 score antm: -85643944917.97


# ASII

In [10]:
idx_end_asii = len(ylist_asii)
idx_start_asii = idx_end_asii - total_period

X_new_asii = []
y_new_asii = []

while idx_start_asii > 0:
  x_line_asii = ylist_asii[idx_start_asii:idx_start_asii+n_past]
  y_line_asii = ylist_asii[idx_start_asii+n_past:idx_start_asii+total_period]

  X_new_asii.append(x_line_asii)
  y_new_asii.append(y_line_asii)

  idx_start_asii = idx_start_asii - 1

X_new_asii = np.array(X_new_asii)
y_new_asii = np.array(y_new_asii)

# train test split
from sklearn.model_selection import train_test_split
X_train_asii, X_test_asii, y_train_asii, y_test_asii = train_test_split(X_new_asii, y_new_asii, test_size=0.33, random_state=42)

# reshape data into the right format for RNNs
n_samples = X_train_asii.shape[0]
n_timesteps = X_train_asii.shape[1]
n_steps = y_train_asii.shape[1]
n_features = 1

X_train_rs_asii = X_train_asii.reshape(n_samples, n_timesteps, n_features )
X_test_rs_asii = X_test_asii.reshape(X_test_asii.shape[0], n_timesteps, n_features )


In [31]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_asii_unit64_lr1_sigmoid = Sequential([
  LSTM(64, activation='sigmoid',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(64, activation='sigmoid',return_sequences=True),
  LSTM(64, activation='sigmoid'),
  Dense(y_train_asii.shape[1]),
])

simple_model_three_asii_unit64_lr1_sigmoid.summary()

simple_model_three_asii_unit64_lr1_sigmoid.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_asii_unit64_lr1_sigmoid = simple_model_three_asii_unit64_lr1_sigmoid.fit(X_train_rs_asii, y_train_asii,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_three_asii_unit64_lr1_sigmoid = simple_model_three_asii_unit64_lr1_sigmoid.predict(X_test_rs_asii)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 60, 64)            16896     
                                                                 
 lstm_7 (LSTM)               (None, 60, 64)            33024     
                                                                 
 lstm_8 (LSTM)               (None, 64)                33024     
                                                                 
 dense_2 (Dense)             (None, 20)                1300      
                                                                 
Total params: 84,244
Trainable params: 84,244
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 9s 86ms/step - loss: 5566.6763 - mae: 5566.6763 - val_loss: 5427.3789 - val_mae: 5427.3789
Epoch 2/20
51/51 [=====================

In [32]:
from sklearn.metrics import mean_absolute_percentage_error, r2_score

print("mae score asii: "+str(mean_absolute_error(preds_three_asii_unit64_lr1_sigmoid, y_test_asii).round(2)))
print("mape score asii: "+str(mean_absolute_percentage_error(preds_three_asii_unit64_lr1_sigmoid, y_test_asii).round(2)))
print("r2 score asii: "+str(r2_score(preds_three_asii_unit64_lr1_sigmoid, y_test_asii).round(2)))


mae score asii: 2053.26
mape score asii: 0.57
r2 score asii: -15025393258.85


# ICBP

In [11]:
idx_end_icbp = len(ylist_icbp)
idx_start_icbp = idx_end_icbp - total_period

X_new_icbp = []
y_new_icbp = []

while idx_start_icbp > 0:
  x_line_icbp = ylist_icbp[idx_start_icbp:idx_start_icbp+n_past]
  y_line_icbp = ylist_icbp[idx_start_icbp+n_past:idx_start_icbp+total_period]

  X_new_icbp.append(x_line_icbp)
  y_new_icbp.append(y_line_icbp)

  idx_start_icbp = idx_start_icbp - 1

X_new_icbp = np.array(X_new_icbp)
y_new_icbp = np.array(y_new_icbp)

# train test split
from sklearn.model_selection import train_test_split
X_train_icbp, X_test_icbp, y_train_icbp, y_test_icbp = train_test_split(X_new_icbp, y_new_icbp, test_size=0.33, random_state=42)

# reshape data into the right format for RNNs
n_samples = X_train_icbp.shape[0]
n_timesteps = X_train_icbp.shape[1]
n_steps = y_train_icbp.shape[1]
n_features = 1

X_train_rs_icbp = X_train_icbp.reshape(n_samples, n_timesteps, n_features )
X_test_rs_icbp = X_test_icbp.reshape(X_test_icbp.shape[0], n_timesteps, n_features )


In [34]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_icbp_unit64_lr1_sigmoid = Sequential([
  LSTM(64, activation='sigmoid',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(64, activation='sigmoid',return_sequences=True),
  LSTM(64, activation='sigmoid'),
  Dense(y_train_icbp.shape[1]),
])

simple_model_three_icbp_unit64_lr1_sigmoid.summary()

simple_model_three_icbp_unit64_lr1_sigmoid.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_icbp_unit64_lr1_sigmoid = simple_model_three_icbp_unit64_lr1_sigmoid.fit(X_train_rs_icbp, y_train_icbp,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_three_icbp_unit64_lr1_sigmoid = simple_model_three_icbp_unit64_lr1_sigmoid.predict(X_test_rs_icbp)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 60, 64)            16896     
                                                                 
 lstm_10 (LSTM)              (None, 60, 64)            33024     
                                                                 
 lstm_11 (LSTM)              (None, 64)                33024     
                                                                 
 dense_3 (Dense)             (None, 20)                1300      
                                                                 
Total params: 84,244
Trainable params: 84,244
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 8s 100ms/step - loss: 6124.2354 - mae: 6124.2354 - val_loss: 5893.0205 - val_mae: 5893.0205
Epoch 2/20
51/51 [====================

In [35]:
from sklearn.metrics import mean_absolute_percentage_error, r2_score

print("mae score icbp: "+str(mean_absolute_error(preds_three_icbp_unit64_lr1_sigmoid, y_test_icbp).round(2)))
print("mape score icbp: "+str(mean_absolute_percentage_error(preds_three_icbp_unit64_lr1_sigmoid, y_test_icbp).round(2)))
print("r2 score icbp: "+str(r2_score(preds_three_icbp_unit64_lr1_sigmoid, y_test_icbp).round(2)))


mae score icbp: 3485.68
mape score icbp: 1.05
r2 score icbp: -264129180019.85


# JSMR

In [12]:
idx_end_jsmr = len(ylist_jsmr)
idx_start_jsmr = idx_end_jsmr - total_period

X_new_jsmr = []
y_new_jsmr = []

while idx_start_jsmr > 0:
  x_line_jsmr = ylist_jsmr[idx_start_jsmr:idx_start_jsmr+n_past]
  y_line_jsmr = ylist_jsmr[idx_start_jsmr+n_past:idx_start_jsmr+total_period]

  X_new_jsmr.append(x_line_jsmr)
  y_new_jsmr.append(y_line_jsmr)

  idx_start_jsmr = idx_start_jsmr - 1

X_new_jsmr = np.array(X_new_jsmr)
y_new_jsmr = np.array(y_new_jsmr)

# train test split
from sklearn.model_selection import train_test_split
X_train_jsmr, X_test_jsmr, y_train_jsmr, y_test_jsmr = train_test_split(X_new_jsmr, y_new_jsmr, test_size=0.33, random_state=42)

# reshape data into the right format for RNNs
n_samples = X_train_jsmr.shape[0]
n_timesteps = X_train_jsmr.shape[1]
n_steps = y_train_jsmr.shape[1]
n_features = 1

X_train_rs_jsmr = X_train_jsmr.reshape(n_samples, n_timesteps, n_features )
X_test_rs_jsmr = X_test_jsmr.reshape(X_test_jsmr.shape[0], n_timesteps, n_features )


In [37]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_jsmr_unit64_lr1_sigmoid = Sequential([
  LSTM(64, activation='sigmoid',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(64, activation='sigmoid',return_sequences=True),
  LSTM(64, activation='sigmoid'),
  Dense(y_train_jsmr.shape[1]),
])

simple_model_three_jsmr_unit64_lr1_sigmoid.summary()

simple_model_three_jsmr_unit64_lr1_sigmoid.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_jsmr_unit64_lr1_sigmoid = simple_model_three_jsmr_unit64_lr1_sigmoid.fit(X_train_rs_jsmr, y_train_jsmr,
          validation_split=0.2,
          epochs=20,
          batch_size=batch_size,
          shuffle = True
)

preds_three_jsmr_unit64_lr1_sigmoid = simple_model_three_jsmr_unit64_lr1_sigmoid.predict(X_test_rs_jsmr)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 60, 64)            16896     
                                                                 
 lstm_13 (LSTM)              (None, 60, 64)            33024     
                                                                 
 lstm_14 (LSTM)              (None, 64)                33024     
                                                                 
 dense_4 (Dense)             (None, 20)                1300      
                                                                 
Total params: 84,244
Trainable params: 84,244
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
51/51 [==============================] - 8s 88ms/step - loss: 4531.8008 - mae: 4531.8008 - val_loss: 4438.8354 - val_mae: 4438.8354
Epoch 2/20
51/51 [=====================

In [38]:
from sklearn.metrics import mean_absolute_percentage_error, r2_score

print("mae score jsmr: "+str(mean_absolute_error(preds_three_jsmr_unit64_lr1_sigmoid, y_test_jsmr).round(2)))
print("mape score jsmr: "+str(mean_absolute_percentage_error(preds_three_jsmr_unit64_lr1_sigmoid, y_test_jsmr).round(2)))
print("r2 score jsmr: "+str(r2_score(preds_three_jsmr_unit64_lr1_sigmoid, y_test_jsmr).round(2)))


mae score jsmr: 1280.3
mape score jsmr: 0.36
r2 score jsmr: -11664832002.29


# EPOCH

## ANTM

In [13]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_antm_unit64_lr1_sigmoid_50 = Sequential([
  LSTM(64, activation='sigmoid',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(64, activation='sigmoid',return_sequences=True),
  LSTM(64, activation='sigmoid'),
  Dense(y_train_antm.shape[1]),
])

simple_model_three_antm_unit64_lr1_sigmoid_50.summary()

simple_model_three_antm_unit64_lr1_sigmoid_50.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_antm_unit64_lr1_sigmoid_50 = simple_model_three_antm_unit64_lr1_sigmoid_50.fit(X_train_rs_antm, y_train_antm,
          validation_split=0.2,
          epochs=50,
          batch_size=batch_size,
          shuffle = True
)

preds_three_antm_unit64_lr1_sigmoid_50 = simple_model_three_antm_unit64_lr1_sigmoid_50.predict(X_test_rs_antm)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 64)            16896     
                                                                 
 lstm_1 (LSTM)               (None, 60, 64)            33024     
                                                                 
 lstm_2 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 20)                1300      
                                                                 
Total params: 84,244
Trainable params: 84,244
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
51/51 [==============================] - 9s 102ms/step - loss: 996.0273 - mae: 996.0273 - val_loss: 887.8474 - val_mae: 887.8474
Epoch 2/50
51/51 [==========================

In [14]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_antm_unit64_lr1_sigmoid_100 = Sequential([
  LSTM(64, activation='sigmoid',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(64, activation='sigmoid',return_sequences=True),
  LSTM(64, activation='sigmoid'),
  Dense(y_train_antm.shape[1]),
])

simple_model_three_antm_unit64_lr1_sigmoid_100.summary()

simple_model_three_antm_unit64_lr1_sigmoid_100.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_antm_unit64_lr1_sigmoid_100 = simple_model_three_antm_unit64_lr1_sigmoid_100.fit(X_train_rs_antm, y_train_antm,
          validation_split=0.2,
          epochs=100,
          batch_size=batch_size,
          shuffle = True
)

preds_three_antm_unit64_lr1_sigmoid_100 = simple_model_three_antm_unit64_lr1_sigmoid_100.predict(X_test_rs_antm)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 60, 64)            16896     
                                                                 
 lstm_4 (LSTM)               (None, 60, 64)            33024     
                                                                 
 lstm_5 (LSTM)               (None, 64)                33024     
                                                                 
 dense_1 (Dense)             (None, 20)                1300      
                                                                 
Total params: 84,244
Trainable params: 84,244
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
51/51 [==============================] - 6s 76ms/step - loss: 996.7971 - mae: 996.7971 - val_loss: 890.2014 - val_mae: 890.2014
Epoch 2/100
51/51 [=======================

## ASII

In [15]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_asii_unit64_lr1_sigmoid_50 = Sequential([
  LSTM(64, activation='sigmoid',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(64, activation='sigmoid',return_sequences=True),
  LSTM(64, activation='sigmoid'),
  Dense(y_train_asii.shape[1]),
])

simple_model_three_asii_unit64_lr1_sigmoid_50.summary()

simple_model_three_asii_unit64_lr1_sigmoid_50.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_asii_unit64_lr1_sigmoid_50_50 = simple_model_three_asii_unit64_lr1_sigmoid_50.fit(X_train_rs_asii, y_train_asii,
          validation_split=0.2,
          epochs=50,
          batch_size=batch_size,
          shuffle = True
)

preds_three_asii_unit64_lr1_sigmoid_50 = simple_model_three_asii_unit64_lr1_sigmoid_50.predict(X_test_rs_asii)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 60, 64)            16896     
                                                                 
 lstm_7 (LSTM)               (None, 60, 64)            33024     
                                                                 
 lstm_8 (LSTM)               (None, 64)                33024     
                                                                 
 dense_2 (Dense)             (None, 20)                1300      
                                                                 
Total params: 84,244
Trainable params: 84,244
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
51/51 [==============================] - 7s 76ms/step - loss: 5570.9395 - mae: 5570.9395 - val_loss: 5437.0625 - val_mae: 5437.0625
Epoch 2/50
51/51 [=====================

In [16]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_asii_unit64_lr1_sigmoid_100 = Sequential([
  LSTM(64, activation='sigmoid',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(64, activation='sigmoid',return_sequences=True),
  LSTM(64, activation='sigmoid'),
  Dense(y_train_asii.shape[1]),
])

simple_model_three_asii_unit64_lr1_sigmoid_100.summary()

simple_model_three_asii_unit64_lr1_sigmoid_100.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_asii_unit64_lr1_sigmoid_100_100 = simple_model_three_asii_unit64_lr1_sigmoid_100.fit(X_train_rs_asii, y_train_asii,
          validation_split=0.2,
          epochs=100,
          batch_size=batch_size,
          shuffle = True
)

preds_three_asii_unit64_lr1_sigmoid_100 = simple_model_three_asii_unit64_lr1_sigmoid_100.predict(X_test_rs_asii)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 60, 64)            16896     
                                                                 
 lstm_10 (LSTM)              (None, 60, 64)            33024     
                                                                 
 lstm_11 (LSTM)              (None, 64)                33024     
                                                                 
 dense_3 (Dense)             (None, 20)                1300      
                                                                 
Total params: 84,244
Trainable params: 84,244
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
51/51 [==============================] - 6s 75ms/step - loss: 5565.2729 - mae: 5565.2729 - val_loss: 5424.6636 - val_mae: 5424.6636
Epoch 2/100
51/51 [===================

## ICBP

In [17]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_icbp_unit64_lr1_sigmoid_50 = Sequential([
  LSTM(64, activation='sigmoid',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(64, activation='sigmoid',return_sequences=True),
  LSTM(64, activation='sigmoid'),
  Dense(y_train_icbp.shape[1]),
])

simple_model_three_icbp_unit64_lr1_sigmoid_50.summary()

simple_model_three_icbp_unit64_lr1_sigmoid_50.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_icbp_unit64_lr1_sigmoid_50_50 = simple_model_three_icbp_unit64_lr1_sigmoid_50.fit(X_train_rs_icbp, y_train_icbp,
          validation_split=0.2,
          epochs=50,
          batch_size=batch_size,
          shuffle = True
)

preds_three_icbp_unit64_lr1_sigmoid_50 = simple_model_three_icbp_unit64_lr1_sigmoid_50.predict(X_test_rs_icbp)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 60, 64)            16896     
                                                                 
 lstm_13 (LSTM)              (None, 60, 64)            33024     
                                                                 
 lstm_14 (LSTM)              (None, 64)                33024     
                                                                 
 dense_4 (Dense)             (None, 20)                1300      
                                                                 
Total params: 84,244
Trainable params: 84,244
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
51/51 [==============================] - 8s 94ms/step - loss: 6130.6235 - mae: 6130.6235 - val_loss: 5906.0146 - val_mae: 5906.0146
Epoch 2/50
51/51 [=====================

In [18]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_icbp_unit64_lr1_sigmoid_100 = Sequential([
  LSTM(64, activation='sigmoid',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(64, activation='sigmoid',return_sequences=True),
  LSTM(64, activation='sigmoid'),
  Dense(y_train_icbp.shape[1]),
])

simple_model_three_icbp_unit64_lr1_sigmoid_100.summary()

simple_model_three_icbp_unit64_lr1_sigmoid_100.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_icbp_unit64_lr1_sigmoid_100_100 = simple_model_three_icbp_unit64_lr1_sigmoid_100.fit(X_train_rs_icbp, y_train_icbp,
          validation_split=0.2,
          epochs=100,
          batch_size=batch_size,
          shuffle = True
)

preds_three_icbp_unit64_lr1_sigmoid_100 = simple_model_three_icbp_unit64_lr1_sigmoid_100.predict(X_test_rs_icbp)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_15 (LSTM)              (None, 60, 64)            16896     
                                                                 
 lstm_16 (LSTM)              (None, 60, 64)            33024     
                                                                 
 lstm_17 (LSTM)              (None, 64)                33024     
                                                                 
 dense_5 (Dense)             (None, 20)                1300      
                                                                 
Total params: 84,244
Trainable params: 84,244
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
51/51 [==============================] - 6s 76ms/step - loss: 6134.4131 - mae: 6134.4131 - val_loss: 5913.7729 - val_mae: 5913.7729
Epoch 2/100
51/51 [===================

## JSMR

In [19]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_jsmr_unit64_lr1_sigmoid_50 = Sequential([
  LSTM(64, activation='sigmoid',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(64, activation='sigmoid',return_sequences=True),
  LSTM(64, activation='sigmoid'),
  Dense(y_train_jsmr.shape[1]),
])

simple_model_three_jsmr_unit64_lr1_sigmoid_50.summary()

simple_model_three_jsmr_unit64_lr1_sigmoid_50.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_jsmr_unit64_lr1_sigmoid_50_50 = simple_model_three_jsmr_unit64_lr1_sigmoid_50.fit(X_train_rs_jsmr, y_train_jsmr,
          validation_split=0.2,
          epochs=50,
          batch_size=batch_size,
          shuffle = True
)

preds_three_jsmr_unit64_lr1_sigmoid_50 = simple_model_three_jsmr_unit64_lr1_sigmoid_50.predict(X_test_rs_jsmr)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_18 (LSTM)              (None, 60, 64)            16896     
                                                                 
 lstm_19 (LSTM)              (None, 60, 64)            33024     
                                                                 
 lstm_20 (LSTM)              (None, 64)                33024     
                                                                 
 dense_6 (Dense)             (None, 20)                1300      
                                                                 
Total params: 84,244
Trainable params: 84,244
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
51/51 [==============================] - 7s 90ms/step - loss: 4520.0503 - mae: 4520.0503 - val_loss: 4415.2134 - val_mae: 4415.2134
Epoch 2/50
51/51 [=====================

In [20]:
import random
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
random.seed(42)

batch_size = 32
simple_model_three_jsmr_unit64_lr1_sigmoid_100 = Sequential([
  LSTM(64, activation='sigmoid',input_shape=(n_timesteps, n_features),return_sequences=True),
  LSTM(64, activation='sigmoid',return_sequences=True),
  LSTM(64, activation='sigmoid'),
  Dense(y_train_jsmr.shape[1]),
])

simple_model_three_jsmr_unit64_lr1_sigmoid_100.summary()

simple_model_three_jsmr_unit64_lr1_sigmoid_100.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
  loss='mean_absolute_error',
  metrics=[
           'mae',
           ],
)

smod_history_three_jsmr_unit64_lr1_sigmoid_100_100 = simple_model_three_jsmr_unit64_lr1_sigmoid_100.fit(X_train_rs_jsmr, y_train_jsmr,
          validation_split=0.2,
          epochs=100,
          batch_size=batch_size,
          shuffle = True
)

preds_three_jsmr_unit64_lr1_sigmoid_100 = simple_model_three_jsmr_unit64_lr1_sigmoid_100.predict(X_test_rs_jsmr)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_21 (LSTM)              (None, 60, 64)            16896     
                                                                 
 lstm_22 (LSTM)              (None, 60, 64)            33024     
                                                                 
 lstm_23 (LSTM)              (None, 64)                33024     
                                                                 
 dense_7 (Dense)             (None, 20)                1300      
                                                                 
Total params: 84,244
Trainable params: 84,244
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
51/51 [==============================] - 7s 91ms/step - loss: 4534.2334 - mae: 4534.2334 - val_loss: 4443.9463 - val_mae: 4443.9463
Epoch 2/100
51/51 [===================

## MAtriks evaluasi

In [21]:
from sklearn.metrics import mean_absolute_percentage_error, r2_score
print('epoch 50')
print("mae score antm: "+str(mean_absolute_error(preds_three_antm_unit64_lr1_sigmoid_50, y_test_antm).round(2)))
print("mape score antm: "+str(mean_absolute_percentage_error(preds_three_antm_unit64_lr1_sigmoid_50, y_test_antm).round(2)))
print("r2 score antm: "+str(r2_score(preds_three_antm_unit64_lr1_sigmoid_50, y_test_antm).round(2)))


epoch 50
mae score antm: 405.59
mape score antm: 0.45
r2 score antm: -12984294681.83


In [22]:
print("mae score asii: "+str(mean_absolute_error(preds_three_asii_unit64_lr1_sigmoid_50, y_test_asii).round(2)))
print("mape score asii: "+str(mean_absolute_percentage_error(preds_three_asii_unit64_lr1_sigmoid_50, y_test_asii).round(2)))
print("r2 score asii: "+str(r2_score(preds_three_asii_unit64_lr1_sigmoid_50, y_test_asii).round(2)))

mae score asii: 838.63
mape score asii: 0.15
r2 score asii: -8191468492.45


In [23]:
print("mae score icbp: "+str(mean_absolute_error(preds_three_icbp_unit64_lr1_sigmoid_50, y_test_icbp).round(2)))
print("mape score icbp: "+str(mean_absolute_percentage_error(preds_three_icbp_unit64_lr1_sigmoid_50, y_test_icbp).round(2)))
print("r2 score icbp: "+str(r2_score(preds_three_icbp_unit64_lr1_sigmoid_50, y_test_icbp).round(2)))

mae score icbp: 2578.36
mape score icbp: 0.44
r2 score icbp: -26195976059.98


In [24]:
print("mae score jsmr: "+str(mean_absolute_error(preds_three_jsmr_unit64_lr1_sigmoid_50, y_test_jsmr).round(2)))
print("mape score jsmr: "+str(mean_absolute_percentage_error(preds_three_jsmr_unit64_lr1_sigmoid_50, y_test_jsmr).round(2)))
print("r2 score jsmr: "+str(r2_score(preds_three_jsmr_unit64_lr1_sigmoid_50, y_test_jsmr).round(2)))

mae score jsmr: 874.42
mape score jsmr: 0.19
r2 score jsmr: -176745391530.0


In [25]:
print('epoch 100')
print("mae score antm: "+str(mean_absolute_error(preds_three_antm_unit64_lr1_sigmoid_100, y_test_antm).round(2)))
print("mape score antm: "+str(mean_absolute_percentage_error(preds_three_antm_unit64_lr1_sigmoid_100, y_test_antm).round(2)))
print("r2 score antm: "+str(r2_score(preds_three_antm_unit64_lr1_sigmoid_100, y_test_antm).round(2)))


epoch 100
mae score antm: 405.71
mape score antm: 0.45
r2 score antm: -52824657596.16


In [26]:
print("mae score asii: "+str(mean_absolute_error(preds_three_asii_unit64_lr1_sigmoid_100, y_test_asii).round(2)))
print("mape score asii: "+str(mean_absolute_percentage_error(preds_three_asii_unit64_lr1_sigmoid_100, y_test_asii).round(2)))
print("r2 score asii: "+str(r2_score(preds_three_asii_unit64_lr1_sigmoid_100, y_test_asii).round(2)))


mae score asii: 838.63
mape score asii: 0.15
r2 score asii: -10625501934.11


In [27]:
print("mae score icbp: "+str(mean_absolute_error(preds_three_icbp_unit64_lr1_sigmoid_100, y_test_icbp).round(2)))
print("mape score icbp: "+str(mean_absolute_percentage_error(preds_three_icbp_unit64_lr1_sigmoid_100, y_test_icbp).round(2)))
print("r2 score icbp: "+str(r2_score(preds_three_icbp_unit64_lr1_sigmoid_100, y_test_icbp).round(2)))


mae score icbp: 2485.69
mape score icbp: 0.36
r2 score icbp: -10050772263.24


In [28]:
print("mae score jsmr: "+str(mean_absolute_error(preds_three_jsmr_unit64_lr1_sigmoid_100, y_test_jsmr).round(2)))
print("mape score jsmr: "+str(mean_absolute_percentage_error(preds_three_jsmr_unit64_lr1_sigmoid_100, y_test_jsmr).round(2)))
print("r2 score jsmr: "+str(r2_score(preds_three_jsmr_unit64_lr1_sigmoid_100, y_test_jsmr).round(2)))


mae score jsmr: 874.66
mape score jsmr: 0.19
r2 score jsmr: -265214429641.12
